In [ ]:
from flask import Flask, jsonify
import pandas as pd
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # Enable Cross-Origin Resource Sharing

# Load the dataset
file_path = 'aqi.csv'  # Make sure this file is in the same directory
df = pd.read_csv(file_path)

# Preprocessing
df['Count'] = pd.to_numeric(df['Count'], errors='coerce')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date', 'Count'])
df['Month'] = df['Date'].dt.month
df['Season'] = df['Month'].apply(lambda x: 'Winter' if x in [12, 1, 2]
                                 else 'Spring' if x in [3, 4, 5]
                                 else 'Summer' if x in [6, 7, 8]
                                 else 'Autumn')


@app.route('/api/data/<city>/boxplot', methods=['GET'])
def boxplot_data(city):
    """API for BoxPlot"""
    city_data = df[df['City'].str.lower() == city.lower()]
    if city_data.empty:
        return jsonify({"error": f"No data available for city '{city}'"}), 404
    return jsonify({"boxPlotData": city_data['Count'].tolist()})


@app.route('/api/data/<city>/timeseries', methods=['GET'])
def timeseries_data(city):
    """API for TimeSeries"""
    city_data = df[df['City'].str.lower() == city.lower()]
    if city_data.empty:
        return jsonify({"error": f"No data available for city '{city}'"}), 404
    return jsonify({
        "timeSeriesData": {
            "dates": city_data['Date'].dt.strftime('%Y-%m-%d').tolist(),
            "counts": city_data['Count'].tolist()
        }
    })


@app.route('/api/data/<city>/histogram', methods=['GET'])
def histogram_data(city):
    """API for Histogram"""
    city_data = df[df['City'].str.lower() == city.lower()]
    if city_data.empty:
        return jsonify({"error": f"No data available for city '{city}'"}), 404
    return jsonify({"histogramData": city_data['Count'].tolist()})


@app.route('/api/data/<city>/seasonaltrends', methods=['GET'])
def seasonal_trends_data(city):
    """API for Seasonal Trends"""
    city_data = df[df['City'].str.lower() == city.lower()]
    if city_data.empty:
        return jsonify({"error": f"No data available for city '{city}'"}), 404
    monthly_avg = city_data.groupby('Month')['Count'].mean().reset_index()
    return jsonify({
        "seasonalTrendsData": {
            "months": monthly_avg['Month'].tolist(),
            "averages": monthly_avg['Count'].tolist()
        }
    })


@app.route('/api/data/<city>/seasonalbarchart', methods=['GET'])
def seasonal_bar_chart_data(city):
    """API for Seasonal Bar Chart"""
    city_data = df[df['City'].str.lower() == city.lower()]
    if city_data.empty:
        return jsonify({"error": f"No data available for city '{city}'"}), 404
    seasonal_avg = city_data.groupby('Season')['Count'].mean().reset_index()
    return jsonify({
        "seasonalBarChartData": {
            "seasons": seasonal_avg['Season'].tolist(),
            "averages": seasonal_avg['Count'].tolist()
        }
    })


if __name__ == '__main__':
    app.run(port=5000,debug=False)


127.0.0.1 - - [05/Dec/2024 04:42:50] "GET /api/data/Peshawar/boxplot HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 04:42:50] "GET /api/data/Peshawar/timeseries HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 04:42:50] "GET /api/data/Peshawar/histogram HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 04:42:50] "GET /api/data/Peshawar/seasonaltrends HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 04:42:50] "GET /api/data/Peshawar/seasonalbarchart HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 04:43:03] "GET /api/data/Lahore/boxplot HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 04:43:03] "GET /api/data/Lahore/timeseries HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 04:43:03] "GET /api/data/Lahore/histogram HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 04:43:03] "GET /api/data/Lahore/seasonaltrends HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 04:43:03] "GET /api/data/Lahore/boxplot HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 04:43:03] "GET /api/data/Lahore/seasonalbarchart HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 04:43:03